## Sample types:
### 'Surf' IQR: 10-10 m
### 'DCM' IQR: 30-75 m
### 515: 515 m
### 170: 170 m

In [83]:
import pandas as pd
import numpy as np
import plotly_utils as pu
import plotly.express as px
import plotly.graph_objs as go
import dash
import json
from dash import dcc, html
from dash.dependencies import Input, Output
import dash_bootstrap_components as dbc
import os
from dotenv import load_dotenv
load_dotenv()
px.set_mapbox_access_token(os.getenv('MAPBOX_ACCESS_TOKEN'))


In [24]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [25]:
%autoreload 2

# Load data
data_16S = pd.read_csv('NCOG_21_16S_redo2_asv_count_tax.tsv', sep='\t')
data_18Sv4 = pd.read_csv('NCOG_18sV4_asv_count_tax.tsv', sep='\t')
data_18Sv9 = pd.read_csv('NCOG_18sV9_asv_count_tax_S.tsv', sep='\t')
data_meta = pd.read_csv('NCOG_sample_log_DNA_stvx_meta_2014-2020_mod.tsv', sep='\t')

# Lat and Lon center for plots
cal_coast_center = dict(
    lat=np.mean([min(data_meta['Lat_Dec']), max(data_meta['Lat_Dec'])]),
    lon=np.mean([min(data_meta['Lon_Dec']), max(data_meta['Lon_Dec'])])
)

# Environmental variables and sample types
env_var_cols = ['T_degC', 'Salnty', 'O2ml_L', 'PO4ug', 'SiO3ug', 'NO3ug', 'NH3ug', 'ChlorA', 'IntC14', 'NCDepth']
sample_type_vals = data_meta['sample_type'].dropna().unique()

# Add X before metadata sampleids to match 16S and 18S sample ids
data_meta['sampleid'] = data_meta['sampleid'].apply(lambda x: 'X' + x)


In [27]:
%%time
map_data = pu.compute_map_data(data_meta)

CPU times: user 60.1 ms, sys: 5.66 ms, total: 65.8 ms
Wall time: 71.2 ms


In [84]:
with open('views/data/map_figure_data.json', 'w') as mapfile:
    mapfile.write(json.dumps(map_data))

In [87]:
with open('views/data/map_figure_data.json', 'r') as mapfile:
    map_data = json.load(mapfile)

In [88]:
%%time
map_figs = pu.precompute_map_figs(map_data)

CPU times: user 670 ms, sys: 5.68 ms, total: 676 ms
Wall time: 676 ms


In [97]:
%%time
sunburst_data = pu.compute_sunburst_fig_data(data_meta, data_16S, data_18Sv4, data_18Sv9,
                                             path=['Phylum', 'Class', 'Order', 'Family'])

CPU times: user 52.6 s, sys: 1.66 s, total: 54.3 s
Wall time: 54.5 s


In [99]:
with open('views/data/sunburst_data.json', 'w') as sunburstfile:
    sunburstfile.write(json.dumps(sunburst_data))

In [100]:
with open('views/data/sunburst_data.json', 'r') as sunburstfile:
    sunburst_data = json.load(sunburstfile)

In [113]:
%%time
sunburst_figs = pu.precompute_sunburst_figs(sunburst_data, path=['Phylum', 'Class', 'Order', 'Family'],
                                            width=700, height=700)


CPU times: user 44.1 s, sys: 1.1 s, total: 45.2 s
Wall time: 45.4 s


In [102]:
%%time
stacked_bar_data, colors_16S, colors_18S = pu.compute_stacked_bar_data(data_meta, data_16S, data_18Sv4,
                                                                       data_18Sv9)


CPU times: user 24.5 s, sys: 1.88 s, total: 26.4 s
Wall time: 25.9 s


In [121]:
with open('views/data/stacked_bar_data.json', 'w') as stackedbarfile:
    stacked_bar_write_data = {
        'stacked_bar_fig_data': stacked_bar_data,
        'colors_16S': colors_16S,
        'colors_18S': colors_18S
    }
    stackedbarfile.write(json.dumps(stacked_bar_write_data))

In [122]:
with open('views/data/stacked_bar_data.json', 'r') as stackedbarfile:
    stacked_bar_data = json.load(stackedbarfile)

In [123]:
%%time
# precompute stacked bar figures
stacked_bar_fig_data, colors_16S, colors_18S = (
    stacked_bar_data['stacked_bar_fig_data'],
    stacked_bar_data['colors_16S'],
    stacked_bar_data['colors_18S']
)
stacked_bar_figs = pu.precompute_stacked_bar_figs(stacked_bar_fig_data, colors_16S, colors_18S)


CPU times: user 1.12 s, sys: 64.1 ms, total: 1.19 s
Wall time: 1.39 s


In [126]:
stacked_bar_figs['093.3 030.0']['16S']

AttributeError: 'Figure' object has no attribute 'keys'

In [107]:
def build_app(app):
    app.layout = dbc.Container([
        html.H1("Marine microbial life at California coastal stations", className="mt-4 mb-4"),
        dbc.Row([
            dbc.Col([
                dbc.Card(
                    dbc.CardBody([
                        html.Div([
                            html.Label('Sample Type:'),
                            dcc.Dropdown(sample_type_vals, 'Surf', id='sample-type-dropdown')
                        ])
                    ])
                )
            ], width=11.5)
        ]),
        dbc.Row([
            dbc.Col([
                dbc.Card(
                    dbc.CardBody([
                        html.Div([
                            html.Label('Environmental Variable (Color):'),
                            dcc.Dropdown(['Total Number of Samples'] + env_var_cols, 'Total Number of Samples', id='env-var-dropdown')
                        ], className="env-var-dropdown")
                    ])
                ),
                dbc.Card(
                    dbc.CardBody([
                        html.Div([
                            dcc.Graph(id='map-graph')
                        ], className="map-graph"),
                    ]),
                ),
                dbc.Card(
                    dbc.CardBody([
                        html.Div([
                            html.Label('Circos Dataset:'),
                            dcc.Dropdown(['16S & 18Sv4', '16S & 18Sv9'], '16S & 18Sv4', id='circos-dropdown')
                        ]),
                        html.Div(id='circos-container', style={'margin-top': '2vw'}),
                    ])
                ),
            ], width=6),
            dbc.Col([
                dbc.Card(
                    dbc.CardBody([
                        html.Div([
                            html.Label('Dataset:'),
                            dcc.Dropdown(['16S', '18Sv4', '18Sv9'], '16S', id='dataset-dropdown')
                        ]),
                    ])
                ),
                dbc.Card(
                    dbc.CardBody([
                        html.Div([
                            html.Button('Reset View', id='reset-button')
                        ]),
                        html.Div([
                            dcc.Graph(id='sunburst-graph')
                        ]),
                    ])
                ),
                dbc.Card(
                    dbc.CardBody([
                        html.Div([
                            dcc.Graph(id='stacked-bar-graph')
                        ])
                    ])
                )
            ], width=6)
        ])
    ], fluid=True)


    # Map graph dropdown callback
    @app.callback(
        Output('map-graph', 'figure'),
        [Input('map-graph', 'clickData'),
        Input('sample-type-dropdown', 'value'),
        Input('env-var-dropdown', 'value')]
    )
    def update_map(click_data, dropdown_sample_type, dropdown_env_var):
        if click_data is None:
            station_id = data_meta['Sta_ID'].iloc[0]
        # Get station ID from hover data
        else:
            if 'customdata' not in click_data['points'][0]:
                station_id = data_meta['Sta_ID'].iloc[0]
            else:
                station_id = click_data['points'][0]['customdata'][0]
        cur_fig = go.Figure(map_figs[dropdown_sample_type][dropdown_env_var])
        cur_fig.update_layout(
            template='plotly_dark',
            plot_bgcolor= 'rgba(0, 0, 0, 0)',
            paper_bgcolor= 'rgba(0, 0, 0, 0)',
            width=600,
            height=600
        )
        stations = cur_fig.data[0]['customdata']
        stations = [x[0] for x in stations]
        clicked_station_idx = stations.index(station_id)
        cur_fig.data[0]['marker']['size'][clicked_station_idx] = 5
        return cur_fig
        

    # Map graph click data callback for sunburst
    @app.callback(
        Output('sunburst-graph', 'figure'),
        [Input('sample-type-dropdown', 'value'),
        Input('map-graph', 'clickData'),
        Input('dataset-dropdown', 'value'),
        Input('reset-button', 'n_clicks')]
    )
    def update_sunburst(dropdown_sample_type, click_data, dropdown_dataset, n_clicks):
        if click_data is None:
            station_id = data_meta['Sta_ID'].iloc[0]
        # Get station ID from hover data
        else:
            if 'customdata' not in click_data['points'][0]:
                station_id = data_meta['Sta_ID'].iloc[0]
            else:
                station_id = click_data['points'][0]['customdata'][0]
        return sunburst_figs[station_id][dropdown_sample_type][dropdown_dataset].update_layout(
            template='plotly_dark',
            plot_bgcolor= 'rgba(0, 0, 0, 0)',
            paper_bgcolor= 'rgba(0, 0, 0, 0)',
            height=800,
            width=800
        )

    # Stacked bar callback
    @app.callback(
        Output('stacked-bar-graph', 'figure'),
        [Input('map-graph', 'clickData'),
        Input('dataset-dropdown', 'value')]
    )
    def update_stacked_bar(click_data, dropdown_dataset):
        if click_data is None:
            station_id = data_meta['Sta_ID'].iloc[0]
        # Get station ID from hover data
        else:
            if 'customdata' not in click_data['points'][0]:
                station_id = station_id = data_meta['Sta_ID'].iloc[0]
            else:
                station_id = click_data['points'][0]['customdata'][0]
        return stacked_bar_figs[station_id][dropdown_dataset].update_layout(
            template='plotly_dark',
            plot_bgcolor= 'rgba(0, 0, 0, 0)',
            paper_bgcolor= 'rgba(0, 0, 0, 0)',
            height=600,
            width=800
        )
    @app.callback(
        Output('circos-container', 'children'),
        [Input('circos-dropdown', 'value')]
    )
    def update_circos(circos_dropdown):
        if circos_dropdown == '16S & 18Sv4':
            src_path = 'assets/NCOG_network_v4_chordDiagram_positive.png'
        elif circos_dropdown == '16S & 18Sv9':
            src_path = 'assets/NCOG_network_v9_chordDiagram_positive.png'
        return html.Img(src=src_path, style={
            'template':'plotly_dark',
            'plot_bgcolor':'rgba(0, 0, 0, 0)',
            'paper_bgcolor':'rgba(0, 0, 0, 0)',
            'height':600,
            'width':600
        })

[2024-04-08 18:36:23,568] ERROR in app: Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/Users/michael/anaconda3/lib/python3.10/site-packages/plotly/basedatatypes.py", line 184, in _check_path_in_prop_tree
    obj = obj[p]
  File "/Users/michael/anaconda3/lib/python3.10/site-packages/plotly/basedatatypes.py", line 728, in __getitem__
    prop = BaseFigure._str_to_dict_path(prop)
  File "/Users/michael/anaconda3/lib/python3.10/site-packages/plotly/basedatatypes.py", line 1846, in _str_to_dict_path
    ret = _str_to_dict_path_full(key_path_str)[0]
  File "/Users/michael/anaconda3/lib/python3.10/site-packages/plotly/basedatatypes.py", line 67, in _str_to_dict_path_full
    if len(key_path_str):
TypeError: object of type 'int' has no len()

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/michael/anaconda3/lib/python3.10/site-packages/flask/app.py", line 1455, in wsgi_app
    respon

[2024-04-08 18:36:26,781] ERROR in app: Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/Users/michael/anaconda3/lib/python3.10/site-packages/plotly/basedatatypes.py", line 184, in _check_path_in_prop_tree
    obj = obj[p]
  File "/Users/michael/anaconda3/lib/python3.10/site-packages/plotly/basedatatypes.py", line 728, in __getitem__
    prop = BaseFigure._str_to_dict_path(prop)
  File "/Users/michael/anaconda3/lib/python3.10/site-packages/plotly/basedatatypes.py", line 1846, in _str_to_dict_path
    ret = _str_to_dict_path_full(key_path_str)[0]
  File "/Users/michael/anaconda3/lib/python3.10/site-packages/plotly/basedatatypes.py", line 67, in _str_to_dict_path_full
    if len(key_path_str):
TypeError: object of type 'int' has no len()

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/michael/anaconda3/lib/python3.10/site-packages/flask/app.py", line 1455, in wsgi_app
    respon

[2024-04-08 18:36:35,181] ERROR in app: Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/Users/michael/anaconda3/lib/python3.10/site-packages/plotly/basedatatypes.py", line 184, in _check_path_in_prop_tree
    obj = obj[p]
  File "/Users/michael/anaconda3/lib/python3.10/site-packages/plotly/basedatatypes.py", line 728, in __getitem__
    prop = BaseFigure._str_to_dict_path(prop)
  File "/Users/michael/anaconda3/lib/python3.10/site-packages/plotly/basedatatypes.py", line 1846, in _str_to_dict_path
    ret = _str_to_dict_path_full(key_path_str)[0]
  File "/Users/michael/anaconda3/lib/python3.10/site-packages/plotly/basedatatypes.py", line 67, in _str_to_dict_path_full
    if len(key_path_str):
TypeError: object of type 'int' has no len()

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/michael/anaconda3/lib/python3.10/site-packages/flask/app.py", line 1455, in wsgi_app
    respon

[2024-04-08 18:36:43,713] ERROR in app: Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/Users/michael/anaconda3/lib/python3.10/site-packages/plotly/basedatatypes.py", line 184, in _check_path_in_prop_tree
    obj = obj[p]
  File "/Users/michael/anaconda3/lib/python3.10/site-packages/plotly/basedatatypes.py", line 728, in __getitem__
    prop = BaseFigure._str_to_dict_path(prop)
  File "/Users/michael/anaconda3/lib/python3.10/site-packages/plotly/basedatatypes.py", line 1846, in _str_to_dict_path
    ret = _str_to_dict_path_full(key_path_str)[0]
  File "/Users/michael/anaconda3/lib/python3.10/site-packages/plotly/basedatatypes.py", line 67, in _str_to_dict_path_full
    if len(key_path_str):
TypeError: object of type 'int' has no len()

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/michael/anaconda3/lib/python3.10/site-packages/flask/app.py", line 1455, in wsgi_app
    respon

In [82]:
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.SLATE])
port=8095
build_app(app)
app.run_server(host='0.0.0.0', port=port)